# PCA

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split

In [57]:
class PCA:
    def __init__(self, n_components, whiten, random_state=41, gram_iter=100):
        self.n_components = n_components # 추출할 주성분의 수
        self.whiten = whiten   # Whitening 적용 여부
        self.gram_iter = gram_iter   # Gram-Schmidt 과정 반복 횟수
        
    def covariance(self, data):
            # 데이터의 평균을 계산
            mean = np.mean(data, axis=0)  # 각 열(변수)의 평균 계산
            centered_data = data - mean    # 데이터 중심화 (평균을 빼줌)
            
            # 공분산 행렬 계산
            cov = np.dot(centered_data.T, centered_data) / (centered_data.shape[0] - 1)

            return cov


    def whitening(self, data, eigvalues=None):
        """
        Whitening 처리
        :param data: 입력 데이터
        :param eigvalues: 고유값 (Whitening 후 처리에 사용)
        :return: Whitening 처리된 데이터
        """
        if eigvalues is None:
            # PCA 전에 데이터를 표준화 (평균을 0으로, 표준편차로 나누기)
            data = data - np.mean(data, axis=0)  # 평균을 0으로
            data = data / np.std(data, axis=0)   # 표준편차로 나누기
            return data
        else:
            # PCA 후 고유값을 사용한 Whitening (고유값으로 스케일링)
            return data / np.sqrt(eigvalues[:self.n_components] + 1e-5) 


    def gram_schmidt(self, A):
        """Gram-Schmidt 과정을 통해 직교 기저 벡터를 구하고 R을 A와 Q를 이용해 계산."""
        n, m = A.shape  # n: 행의 수, m: 열의 수
        Q = np.zeros((n, m))  # 직교 기저 벡터를 저장할 행렬
        R = np.zeros((m, m))  # R 행렬 초기화

        for j in range(m):
            v = A[:, j]  # A의 j번째 열 벡터를 v로 설정
            #print('gram schmidt iter : ', j ,'/', m , end = '\r')
            # 직교화 과정
            for i in range(j):
                q = Q[:, i]  # 직교 기저 벡터 Q의 i번째 열 벡터
                v = v - (np.dot(q, v)) * q  # v에서 q 방향으로의 성분을 제거
            
            # 정규화
            norm_v = np.linalg.norm(v)
            if norm_v > 1e-10:  # 0으로 나누는 것을 방지
                Q[:, j] = v / norm_v
            else:
                #print(f"경고: {j}번째 벡터는 선형 종속적이므로 추가 x.")
                Q[:, j] = np.zeros(n)  # 0 벡터 추가

        # R을 Q와 A를 사용하여 계산
        R = np.dot(Q.T, A)  # R = Q^T A

        return Q, R

    def eig(self, data):
        "Av 백터 행렬이 있을때 kV에서 K는 A가 상수배면 K는 고유값 V는 고유벡터라 함."
        # 공분산 행렬 계산
        cov_matrix = self.covariance(data)
        
        # 초기화
        A = cov_matrix.copy()  # 공분산 행렬 복사본
        n = A.shape[0]  # 행렬의 크기
        Q_total = np.eye(n)  # 초기 Q_total은 단위행렬
        
        # 수렴 조건 설정 (tol: 수렴 허용 오차)
        tol = 1e-6  # 대각 성분 외 값이 이 값 이하로 작아지면 수렴으로 간주

        for i in range(self.gram_iter):
            # Gram-Schmidt 과정으로 직교 행렬 Q 구하기
            Q, R = self.gram_schmidt(A)

            # A를 R * Q로 업데이트 (QR 알고리즘)
            A_new = np.dot(R, Q)

            # 전체 Q는 각 단계의 Q의 곱으로 업데이트
            Q_total = np.dot(Q_total, Q)

            # 대각 성분 외의 값들이 모두 거의 0으로 수렴했는지 확인
            off_diagonal = A_new - np.diag(np.diag(A_new))  # 대각 성분 제외한 행렬
            max_off_diagonal = np.max(np.abs(off_diagonal))  # 대각 성분 제외한 최대값 계산

            if max_off_diagonal < tol:
                print(f"{i + 1}번째 반복에서 수렴. 대각 성분 외 최대값: {max_off_diagonal}")
                break

            # A 업데이트
            A = A_new

            # 진행 상황 출력
            print(f"QR 분해 반복: {i + 1}/{self.gram_iter} 완료, 대각 성분 외 최대값: {max_off_diagonal}")

        # 고유값과 고유벡터 계산
        eigvalues = np.diag(A)
        eigvectors = Q_total

        # 고유값과 고유벡터를 고유값 크기 순서대로 정렬
        idx = np.argsort(eigvalues)[::-1]
        eigvalues = eigvalues[idx]
        eigvectors = eigvectors[:, idx]

        return eigvalues, eigvectors
    
    def fit(self, data):
        """
        주성분 분석(PCA)을 수행하여 데이터를 변환
        """
        if self.whiten:
            # 1. Whitening 적용 (평균을 0으로 하고 표준편차로 나누는 작업 포함)
            pca_output = self.whitening(data)
        else:
            # 1. 데이터를 평균 제거한 후 공분산 행렬 계산
            pca_output = data - np.mean(data, axis=0)  # 평균 제거
        
        # 2. 고유값과 고유벡터 계산 (QR 분해 수렴 고려)
        eigvalues, eigvectors = self.eig(pca_output)
        #print("고유값",eigvalues)
        #print("고유벡터",eigvectors)
        
        # 3. 상위 n_components개의 고유벡터 선택
        selected_vectors = eigvectors[:, :self.n_components]  # n_components만큼 고유벡터 선택
        
        # 4. 데이터를 선택한 고유벡터로 변환 (고유벡터 공간으로 투영)
        pca_output = np.dot(pca_output, selected_vectors)
        
        """(추가 개선한 코드): 마지막에 고유값으로 스케일링 함으로써 정보 손실을 낮춤 보고서에 추가로 작성 """
        if self.whiten:
            # Whitening 적용 시 고유값으로 스케일 조정
            pca_output = self.whitening(pca_output, eigvalues)
        
        return pca_output

In [58]:
"""인공지능 강의자료에 있는 데이터를 실험하여 정확하게 나오는지 비교하는 테스트 케이스"""

# 테스트 데이터 생성 (2차원 데이터)
data = np.array([[2.5000, 2.4000],
                 [0.5000, 0.7000],
                 [2.2000, 2.9000],
                 [1.9000, 2.2000],
                 [3.1000, 3.0000],
                 [2.3000, 2.7000],
                 [2.0000, 1.6000],
                 [1.0000, 1.1000],
                 [1.5000, 1.6000],
                 [1.1000, 0.9000]])


# PCA 클래스 인스턴스 생성 (n_components=1로 설정, Whitening은 적용하지 않음)
pca = PCA(n_components=2, whiten= False)
dataC=data-np.mean(data, axis=0)
# PCA fit 메서드 실행 (차원 축소 수행)
eigvalues, eigvectors = pca.eig(dataC)
pca_output = pca.fit(data)
print("고유값" ,eigvalues)
print("고유벡터 ",eigvectors)
# 결과 출력
print("원본 데이터:")
print(data)
print("\nPCA 변환된 데이터 (1차원):")
print(pca_output)

QR 분해 반복: 1/100 완료, 대각 성분 외 최대값: 0.0511100267858857
QR 분해 반복: 2/100 완료, 대각 성분 외 최대값: 0.0019570911219078265
QR 분해 반복: 3/100 완료, 대각 성분 외 최대값: 7.481219002244574e-05
QR 분해 반복: 4/100 완료, 대각 성분 외 최대값: 2.85977985288437e-06
5번째 반복에서 수렴. 대각 성분 외 최대값: 1.0931829126737184e-07
QR 분해 반복: 1/100 완료, 대각 성분 외 최대값: 0.0511100267858857
QR 분해 반복: 2/100 완료, 대각 성분 외 최대값: 0.0019570911219078265
QR 분해 반복: 3/100 완료, 대각 성분 외 최대값: 7.481219002244574e-05
QR 분해 반복: 4/100 완료, 대각 성분 외 최대값: 2.85977985288437e-06
5번째 반복에서 수렴. 대각 성분 외 최대값: 1.0931829126737184e-07
고유값 [1.28402771 0.0490834 ]
고유벡터  [[ 0.67787346 -0.7351786 ]
 [ 0.7351786   0.67787346]]
원본 데이터:
[[2.5 2.4]
 [0.5 0.7]
 [2.2 2.9]
 [1.9 2.2]
 [3.1 3. ]
 [2.3 2.7]
 [2.  1.6]
 [1.  1.1]
 [1.5 1.6]
 [1.1 0.9]]

PCA 변환된 데이터 (1차원):
[[ 0.8279702  -0.17511523]
 [-1.77758034  0.14285707]
 [ 0.99219746  0.38437508]
 [ 0.2742104   0.13041723]
 [ 1.67580144 -0.20949831]
 [ 0.91294909  0.17528252]
 [-0.09910941 -0.34982471]
 [-1.14457217  0.04641716]
 [-0.43804614  0.01776459]

In [6]:
"""Gram_Schmidt 알고리즘을 사용한 것에 대한 Q 와 R이 정확히 구성돠어있는지 확인하는 테스트 케이스스"""
def test_gram_schmidt():
    # 랜덤한 5x3 행렬 생성
    A = np.random.rand(62, 47)  # 0에서 1 사이의 랜덤 값으로 채운 62x47 행렬
    
    
    # PCA 객체 생성
    pca = PCA(n_components=3, whiten=False)
    #AB=pca.covariance(A)
    
    # Gram-Schmidt 과정 수행
    Q, R = pca.gram_schmidt(A)

    # Q의 직교성 확인
    for i in range(Q.shape[1]):
        for j in range(i + 1, Q.shape[1]):
            assert np.isclose(np.dot(Q[:, i], Q[:, j]), 0, atol=1e-10), f"Q의 {i}와 {j} 벡터는 직교하지 않습니다."
    
    # Q의 정규화 확인
    for i in range(Q.shape[1]):
        assert np.isclose(np.linalg.norm(Q[:, i]), 1, atol=1e-10), f"Q의 {i} 벡터는 정규화되어 있지 않습니다."

    # A가 Q와 R로 재구성되는지 확인
    A_reconstructed = np.dot(Q, R)
    assert np.allclose(A, A_reconstructed, atol=1e-10), "A가 Q와 R로 재구성되지 않습니다."

    print("Gram-Schmidt 테스트가 성공적으로 완료되었습니다!")

# 테스트 실행
test_gram_schmidt()

Gram-Schmidt 테스트가 성공적으로 완료되었습니다!


In [59]:
""" 그램 슈미츠 알고리즘과 내장되어있는 QR 함수와에 Q와 R을 비교하는 테스트 케이스"""
A = np.random.rand(3,2) 
# PCA 클래스의 인스턴스 생성
pca = PCA(0,whiten=False)
AB=pca.covariance(A)

# Gram-Schmidt 과정 실행
Q,R= pca.gram_schmidt(AB)

print("직접 구현한 Q:")
print(Q)

# 직교성 확인
print("\nQ^T * Q:")
print(np.dot(Q.T, Q))

print("R 값")
print(R)

RQ,RR=np.linalg.qr(AB)
print("정답 Q행렬 ")
print(RQ)

print("정답 R행렬")
print(RR)



직접 구현한 Q:
[[ 0.81431933 -0.58041712]
 [ 0.58041712  0.81431933]]

Q^T * Q:
[[ 1.0000000e+00 -1.6271364e-17]
 [-1.6271364e-17  1.0000000e+00]]
R 값
[[ 1.49109431e-02  2.09444763e-02]
 [-2.22208936e-19  1.44739443e-02]]
정답 Q행렬 
[[-0.81431933 -0.58041712]
 [-0.58041712  0.81431933]]
정답 R행렬
[[-0.01491094 -0.02094448]
 [ 0.          0.01447394]]


In [8]:
"""구현한 EIG 함수를 내장되어있는 EIG 함수와 비교히여 고유벡터와 고유값이 정확한지 비교하는 테스트"""
def test_eig():
    # 랜덤한 정방향 행렬 생성 (10x5 크기)
    # 테스트 데이터 생성 (2차원 데이터)
# 테스트 데이터 생성 (2차원 데이터)
    data = np.array([[2.5000, 2.4000],
                 [0.5000, 0.7000],
                 [2.2000, 2.9000],
                 [1.9000, 2.2000],
                 [3.1000, 3.0000],
                 [2.3000, 2.7000],
                 [2.0000, 1.6000],
                 [1.0000, 1.1000],
                 [1.5000, 1.6000],
                 [1.1000, 0.9000]])
    dataC=data-np.mean(data, axis=0)

    # PCA 객체 생성
    pca = PCA(0, False)
    covv = pca.covariance(dataC)

    # 고유값 및 고유벡터 계산
    eigvalues, eigvectors = pca.eig(dataC)
    print("고유값:", eigvalues)
    #print("고유 벡터:\n", eigvectors)

    # 고유값이 올바른지 확인 (양수여야 함)
    assert np.all(eigvalues >= 0), "고유값이 0보다 작거나 같습니다."

    # 고유벡터의 형상 확인
    #assert eigvectors.shape == (data.shape[1], data.shape[1]), "고유벡터의 형상이 잘못되었습니다."

    # 대각 성분 (고유값)이 실제 공분산 행렬의 고유값과 일치하는지 검증
    eigen_decomp = np.linalg.eig(covv)
    expected_eigvalues = np.sort(eigen_decomp[0])[::-1]  # 내림차순 정렬
    expected_eigvectors = eigen_decomp[1][:, np.argsort(eigen_decomp[0])[::-1]]  # 고유값 정렬에 따라 고유벡터 정렬

    print("고유값 정답:", expected_eigvalues)
    
    # 허용 오차를 조정하여 assert 문 적용
    assert np.allclose(eigvalues, expected_eigvalues, atol=1e-4), "고유값이 예상과 일치하지 않습니다."

    #print("고유 벡터 정답:\n", expected_eigvectors)

    # 고유벡터 비교: 부호를 무시하고 비교
    for i in range(expected_eigvectors.shape[1]):
        comparison_result = np.allclose(eigvectors[:, i], expected_eigvectors[:, i], atol=1e-4) or np.allclose(eigvectors[:, i], -expected_eigvectors[:, i], atol=1e-5)
        assert comparison_result, f"{i}번째 고유벡터가 예상과 일치하지 않습니다."

    print("테스트가 성공적으로 완료되었습니다!")

# 테스트 실행
test_eig()

QR 분해 반복: 1/100 완료, 대각 성분 외 최대값: 0.0511100267858857
QR 분해 반복: 2/100 완료, 대각 성분 외 최대값: 0.0019570911219078265
QR 분해 반복: 3/100 완료, 대각 성분 외 최대값: 7.481219002244574e-05
QR 분해 반복: 4/100 완료, 대각 성분 외 최대값: 2.85977985288437e-06
5번째 반복에서 수렴. 대각 성분 외 최대값: 1.0931829126737184e-07
고유값: [1.28402771 0.0490834 ]
고유값 정답: [1.28402771 0.0490834 ]
테스트가 성공적으로 완료되었습니다!


In [56]:
"""공분산 함수와 직접 구현한 공분산 함수와 비교 하는 테스트 만약 둘에 차가 0이면 실제 공분산이 올바름을 인식"""

# 데이터셋 가져오기
people = fetch_lfw_people(min_faces_per_person=100, resize=0.5)

# 데이터 행렬 출력
data = people.data  # 데이터 행렬


# PCA 객체 생성
pca = PCA(0, False)

data = np.random.randn(5, 10)
print("data shape:", data.shape)  # 데이터의 형태 출력

# 직접 구현한 공분산 행렬 계산
my_cov_matrix = pca.covariance(data)

# NumPy의 np.cov를 사용하여 공분산 행렬 계산 (행 방향으로 계산)
np_cov_matrix = np.cov(data, rowvar=False)  # rowvar=False는 각 열이 변수임을 의미

# 공분산 행렬 출력
#print("My Covariance Matrix:\n", my_cov_matrix)
#print("NumPy Covariance Matrix:\n", np_cov_matrix)

# 두 행렬의 차이 확인
difference = my_cov_matrix - np_cov_matrix
print("Difference:\n", difference)

data shape: (5, 10)
Difference:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [11]:
people = fetch_lfw_people(min_faces_per_person=100, resize=0.5)
image_shape = people.images[0].shape
print("dataset keys        : ", people.keys())
print("dataset.images shape: ", people.images.shape) #원본 이미지 배열
print("dataset.data shape  : ", people.data.shape)
print("dataset.target shape: ", people.target.shape)


dataset keys        :  dict_keys(['data', 'images', 'target', 'target_names', 'DESCR'])
dataset.images shape:  (1140, 62, 47)
dataset.data shape  :  (1140, 2914)
dataset.target shape:  (1140,)


In [12]:
"""스레드를 사용하여 두개를 동시에 돌림"""
import threading as th
n_component_data =None
n_component_data_w=None
def pca_th():
    global n_component_data
    pca = PCA(200, False)
    n_component_data = pca.fit(people.data)
def pca_w_th():
    global n_component_data_w
    pca_w = PCA(200, True)
    n_component_data_w = pca_w.fit(people.data)

th_pca = th.Thread(target=pca_th)
th_pca_w = th.Thread(target=pca_w_th)
th_pca.start()
th_pca_w.start()

th_pca.join()
th_pca_w.join()


X_train, X_test, y_train, y_test = train_test_split(n_component_data, people.target, test_size=0.25, random_state=41)
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(n_component_data_w, people.target, test_size=0.25, random_state=41)

QR 분해 반복: 1/100 완료, 대각 성분 외 최대값: 174.8355553464133
QR 분해 반복: 1/100 완료, 대각 성분 외 최대값: 2.823815360783045
QR 분해 반복: 2/100 완료, 대각 성분 외 최대값: 110.77238580680496
QR 분해 반복: 2/100 완료, 대각 성분 외 최대값: 1.7258642948650482
QR 분해 반복: 3/100 완료, 대각 성분 외 최대값: 70.90892817070734
QR 분해 반복: 3/100 완료, 대각 성분 외 최대값: 0.9381199579851704
QR 분해 반복: 4/100 완료, 대각 성분 외 최대값: 41.74805269445571
QR 분해 반복: 4/100 완료, 대각 성분 외 최대값: 0.6698769325559465
QR 분해 반복: 5/100 완료, 대각 성분 외 최대값: 0.5868341762533094
QR 분해 반복: 5/100 완료, 대각 성분 외 최대값: 24.919596656132676
QR 분해 반복: 6/100 완료, 대각 성분 외 최대값: 18.883411400432646
QR 분해 반복: 6/100 완료, 대각 성분 외 최대값: 0.5640856387014289
QR 분해 반복: 7/100 완료, 대각 성분 외 최대값: 13.704683764698283
QR 분해 반복: 7/100 완료, 대각 성분 외 최대값: 0.5114582036258454
QR 분해 반복: 8/100 완료, 대각 성분 외 최대값: 11.355023654004574
QR 분해 반복: 8/100 완료, 대각 성분 외 최대값: 0.4400549888162663
QR 분해 반복: 9/100 완료, 대각 성분 외 최대값: 9.180249876374427
QR 분해 반복: 9/100 완료, 대각 성분 외 최대값: 0.36405059352846997
QR 분해 반복: 10/100 완료, 대각 성분 외 최대값: 6.85888458886
QR 분해 반복: 10/100 완료,

# KNN

In [19]:
from sklearn.neighbors import KNeighborsClassifier as knc

In [20]:
knn = knc(n_neighbors=1)
knn_w = knc(n_neighbors=1)

In [21]:
knn.fit(X_train, y_train)
knn_w.fit(X_train_w, y_train_w)

KNeighborsClassifier(n_neighbors=1)

In [22]:
y_pred = knn.predict(X_test)
y_pred_w = knn_w.predict(X_test_w)

# Results
#### -F1 score를 각 채점 항목 별로 출력하면 됩니다

In [23]:
from sklearn.metrics import f1_score

In [40]:
print(f"Naive PCA: {f1_score(y_test, y_pred, average='micro')}\nWhitening PCA: {f1_score(y_test_w, y_pred_w, average='micro')}")

Naive PCA: 0.5894736842105263
Whitening PCA: 0.6701754385964912
